# 1) Prompt Chaining for a Customer Support AI

**Goal:** Build a simple prompt chain to simulate a customer service flow.

**What this notebook shows:**
- A lightweight, prompt-engineering-style pipeline where each step has a role, a prompt template, and a tool.
- Deterministic, mock-LLM responses so the notebook runs reliably without external APIs.
- A full, logged transcript of the chain.


### Prompt & Tool Design
**Personas / Steps:**
1. `Greeter` → Welcomes the user and asks a clarifying question.
2. `Classifier` → Classifies the issue category (billing, technical, account, other).
3. `Troubleshooter` → Suggests next steps using a small in-notebook knowledge base.
4. `Summarizer` → Summarizes the session & provides the resolution plan.

**Knowledge Base (toy):** a few entries keyed by category.

**Why a chain?** It keeps prompts small and role-specific, which often yields more reliable behavior than a single, giant prompt.

In [1]:

from dataclasses import dataclass
from typing import Dict, List

# --- Mock tools and knowledge ---
KB: Dict[str, str] = {
    "billing": "Check latest invoice → verify charges → apply credit if error → send receipt.",
    "technical": "Ask for device/OS → have the user reboot app → clear cache → reinstall if needed.",
    "account": "Verify identity → reset password → confirm recovery email → enable 2FA.",
    "other": "Collect details → route to human agent with category tag."
}

# --- Prompt templates for each step ---
PROMPTS = {
    "greeter": (
        "You are a warm, concise support greeter.\n"
        "Task: Greet the customer by name and ask a short clarifying question based on their initial message.\n"
        "Customer name: {name}\n"
        "Customer message: {message}\n"
        "Reply in 1-2 sentences."
    ),
    "classifier": (
        "You are a precise ticket classifier.\n"
        "Task: Categorize the customer's issue as one of: billing, technical, account, other.\n"
        "Return ONLY the category word in lowercase.\n"
        "Customer message: {message}\n"
        "Clarifying detail: {clarification}"
    ),
    "troubleshooter": (
        "You are a practical troubleshooter.\n"
        "Task: Provide 3-5 concrete next steps using the knowledge base guidance for the predicted category.\n"
        "Category: {category}\n"
        "KB guidance: {kb}\n"
        "Customer context: {message} / {clarification}\n"
        "Write numbered steps."
    ),
    "summarizer": (
        "You are a support session summarizer.\n"
        "Task: Summarize the issue and list the agreed next actions in <= 5 bullet points.\n"
        "Customer name: {name}\n"
        "Category: {category}\n"
        "Proposed steps:\n{steps}\n"
        "Be specific but brief."
    )
}

@dataclass
class Message:
    role: str
    content: str

class MockLLM:
    """Deterministic 'LLM' that formats templates and simulates reasonable outputs."""
    def generate(self, template: str, **kwargs) -> str:
        formatted = template.format(**kwargs)
        # A few minimal heuristics to simulate behavior
        if "Return ONLY the category word" in template:
            txt = (kwargs.get("message", "") + " " + kwargs.get("clarification", "")).lower()
            if any(k in txt for k in ("charge", "payment", "invoice", "billing")):
                return "billing"
            if any(k in txt for k in ("error", "bug", "crash", "install", "update", "slow")):
                return "technical"
            if any(k in txt for k in ("login", "password", "account", "2fa")):
                return "account"
            return "other"
        return formatted  # For others, we'll fill in below in the chain logic

class SupportChain:
    def __init__(self):
        self.llm = MockLLM()
        self.transcript: List[Message] = []

    def run(self, name: str, message: str, clarification: str):
        # Step 1: Greeter (we'll craft a response programmatically for reliability)
        greeter_out = f"Hi {name}! Thanks for reaching out. Quick question: {clarification}?"
        self.transcript.append(Message("greeter", greeter_out))

        # Step 2: Classifier
        category = self.llm.generate(PROMPTS["classifier"], message=message, clarification=clarification)
        self.transcript.append(Message("classifier", category))

        # Step 3: Troubleshooter
        kb = KB.get(category, KB["other"])
        # We'll create a structured response as numbered steps.
        arrows = kb.split("→")
        steps = []
        for i, chunk in enumerate(arrows):
            chunk = chunk.strip().rstrip(".")
            label = ["Review", "Then", "Next", "Finally"]
            idx = min(i, len(label)-1)
            steps.append(f"{label[idx]}: {chunk}.")
        steps_text = "\n".join(f"{i+1}. {s}" for i, s in enumerate(steps))
        self.transcript.append(Message("troubleshooter", steps_text))

        # Step 4: Summarizer
        summary = (
            f"- **Issue:** {message}\n"
            f"- **Category:** {category}\n"
            f"- **Next actions:**\n" + "\n".join(f"  - {s[3:]}" for s in steps_text.splitlines())
        )
        self.transcript.append(Message("summarizer", summary))

        return {
            "category": category,
            "steps": steps_text,
            "summary": summary
        }

chain = SupportChain()
result = chain.run(
    name="Khanh",
    message="I keep getting charged twice on my card for last month's plan.",
    clarification="Is this about last month's billing statement"
)

print("=== CATEGORY ===")
print(result["category"])
print("\n=== STEPS ===")
print(result["steps"])
print("\n=== SUMMARY ===")
print(result["summary"])

print("\n--- TRANSCRIPT ---")
for m in chain.transcript:
    print(f"[{m.role.upper()}] {m.content}")


=== CATEGORY ===
billing

=== STEPS ===
1. Review: Check latest invoice.
2. Then: verify charges.
3. Next: apply credit if error.
4. Finally: send receipt.

=== SUMMARY ===
- **Issue:** I keep getting charged twice on my card for last month's plan.
- **Category:** billing
- **Next actions:**
  - Review: Check latest invoice.
  - Then: verify charges.
  - Next: apply credit if error.
  - Finally: send receipt.

--- TRANSCRIPT ---
[GREETER] Hi Khanh! Thanks for reaching out. Quick question: Is this about last month's billing statement?
[CLASSIFIER] billing
[TROUBLESHOOTER] 1. Review: Check latest invoice.
2. Then: verify charges.
3. Next: apply credit if error.
4. Finally: send receipt.
[SUMMARIZER] - **Issue:** I keep getting charged twice on my card for last month's plan.
- **Category:** billing
- **Next actions:**
  - Review: Check latest invoice.
  - Then: verify charges.
  - Next: apply credit if error.
  - Finally: send receipt.
